In [6]:
from functools import partial
import numpy as np
from goodpoints import kt, compress

In [7]:

def kernel_eval(x, y, params_k):
    """Returns matrix of kernel evaluations kernel(xi, yi) for each row index i.
    x and y should have the same number of columns, and x should either have the
    same shape as y or consist of a single row, in which case, x is broadcasted 
    to have the same shape as y.
    """
    if params_k["name"] in ["gauss", "gauss_rt"]:
        k_vals = np.sum((x-y)**2,axis=1)
        scale = -.5/params_k["var"]
        return(np.exp(scale*k_vals))
    
    raise ValueError("Unrecognized kernel name {}".format(params_k["name"]))


In [13]:
delta = 0.5
size = 4
g = 0
# Each Compress Halve call applied to an input of length l uses KT( l^2 * halve_prob ) 
halve_prob = delta / ( 4*(4**size)*(2**g)*( g + (2**g) * (size  - g) ) )
###halve_prob = 0 if size == g else delta * .5 / (4 * (4**size) * (4 ** g) * (size - g) ) ###
# Each Compress++ Thin call uses KT( thin_prob )
thin_prob = delta * g / (g + ( (2**g)*(size - g) ))
d = int(2)
bandwidth = 1.0
var = np.square(float(bandwidth))
params_k_swap = {"name": "gauss", "var": var, "d": int(d)}
params_k_split = {"name": "gauss_rt", "var": var/2., "d": int(d)}
split_kernel = partial(kernel_eval, params_k=params_k_split)
swap_kernel = partial(kernel_eval, params_k=params_k_swap)

thin = partial(kt.thin, m=g, split_kernel = split_kernel, swap_kernel = swap_kernel, delta = thin_prob)
halve = compress.symmetrize(lambda x: kt.thin(X = x, m=1, split_kernel = split_kernel, swap_kernel = swap_kernel, 
                                              unique=True, delta = halve_prob*(len(x)**2)))


X = np.random.randn(1024, 2)
coreset = compress.compresspp(X, halve, thin, g)

In [14]:
coreset.shape

(32,)